# **Column Wise Approach**

### **Installation Libraries**

In [62]:
import io
import fitz
import os
import cv2
import numpy as np


resume_link = '/kaggle/input/resume-dataset/Resumes/DUAL_COLUMN_ENGLISH_DATASETS/Resume129.pdf'
if resume_link.endswith('.pdf'):
    print("File in pdf format")

    doc = fitz.open(resume_link)
#     print(len(doc))

    # Loop through each page in the PDF
    for page_number in range(len(doc)):
       
        page = doc.load_page(page_number)  # Get the page
        pix = page.get_pixmap()  # Convert the page to an image
        if page_number == 0:
            output_image_path = "/kaggle/working/original_image.jpg"
            # Save the image as a JPG file
            pix.save(output_image_path)
        else:
            pix.save(f"/kaggle/working/original_image(Page-{page_number}).jpg")
            print(f"Re-run all below code by changing in image path of YOLO by /kaggle/working/original_image_(Page-{page_number}).jpg")
#     if len(doc) > 1:
#         print("Still only one page resume is taken into account")
    # Close the document
    doc.close()
elif resume_link.endswith('.jpg') or resume_link.endswith('jpeg') or resume_link.endswith('png'):
    print("File in image format")
    img = cv2.imread(resume_link)
    cv2.imwrite('/kaggle/working/original_image.jpg', img)
else:
    print("File format not supported!! [Supported formats: .jpg/.png/.jpeg/.pdf]")
# else: ######have to work on this
#         for page_number in range(len(doc)):
#             page = doc.load_page(page_number)  # Get the page
#             pix = page.get_pixmap()  # Convert the page to an image

#             # Open the images you want to combine
#             o1 = "/kaggle/working/original_image.jpg"
#             if os.path.exists(o1):
#                 print(0)
#                 image1_arr = cv2.imread(o1)  # Image 1 as NumPy array

#                 # Convert pixmap to PIL image
#                 image2_pil = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
#                 image2 = cv2.cvtColor(np.array(image2_pil), cv2.COLOR_RGB2BGR)  # Convert to OpenCV format

#                 # Get dimensions for image1 and image2 using shape (height, width, channels)
#                 h1, w1, _ = image1_arr.shape
#                 h2, w2, _ = image2.shape

#                 # Create a new blank image for vertical combination
#                 new_image_v = np.zeros((h1 + h2, max(w1, w2), 3), dtype=np.uint8)
                
#                 image1 = cv2.cvtColor(np.array(image1_arr), cv2.COLOR_BGR2RGB)
#                 # Paste image1 and image2 onto new_image_v
#                 new_image_v[:h1, :w1] = image1  # Place image1
#                 new_image_v[h1:h1+h2, :w2] = image2  # Place image2 below image1

#                 # Save the combined image
#                 new_image = Image.fromarray(new_image_v)
#                 new_image.save("/kaggle/working/original_image.jpg")

#             else:
#                 print(1)
#                 pix.save(o1)




File in pdf format


### **Removal of photo**

In [63]:
# Perform object detection using YOLOv5
import cv2
image =  "/kaggle/working/original_image.jpg" #If needed change the link for multiple pages
# %store image
im1 = cv2.imread(image)
fixed_size = (640,640)
r_image = cv2.resize(im1, fixed_size)
cv2.imwrite('/kaggle/working/resized_image.jpg', r_image)
#Save results in a custom directory
output =  "/kaggle/working/"
# Perform object detection using YOLOv5
!python detect.py --source '/kaggle/working/resized_image.jpg' --weights yolov5s.pt --img 640 --conf 0.01 --project "{output}" --save-txt --name exp


detect: weights=['yolov5s.pt'], source=/kaggle/working/resized_image.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.01, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/kaggle/working/, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/1 /kaggle/working/resized_image.jpg: 640x640 1 person, 1 laptop, 2 books, 12.2ms
Speed: 0.6ms pre-process, 12.2ms inference, 174.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to /kaggle/working/exp
1 labels saved to /kaggle/working/exp/labels


In [64]:
import torch
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Load the YOLOv5 model (YOLOv5s for small model)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Load the image
im1 = cv2.imread("/kaggle/working/exp/resized_image.jpg")####################

image = cv2.imread("/kaggle/working/resized_image.jpg")#####################

# Read from YOLOv5 output text file
image_width = 640
image_height = 640
output_file = '/kaggle/working/exp/labels/resized_image.txt'  ################ Path to the saved results
x1=x2=y1=y2 = 0.000000
c = 0
if os.path.isfile(output_file):
    with open(output_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            values = line.split()
            class_id = int(values[0])
            if class_id == 0:
                if c == 1:
                    height_norm1 = float(values[4])
                    if height_norm1 > height_norm:
                        break
                c = c + 1
                x_center_norm = float(values[1])
                y_center_norm = float(values[2])
                width_norm = float(values[3])
                height_norm = float(values[4])

                # Convert to pixel coordinates
                x_center = x_center_norm * image_width
                y_center = y_center_norm * image_height
                width = width_norm * image_width
                height = height_norm * image_height

                x1 = x_center - (width / 2)
                y1 = y_center - (height / 2)
                x2 = x_center + (width / 2)
                y2 = y_center + (height / 2)

    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    print(f"Coordinates: x1={x1}, y1={y1}, x2={x2}, y2={y2}")
            
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 255, 255), -1)

# Save or display the result
cv2.imwrite('/kaggle/working/results_person_removed.jpg', image)
#-----------------------------------------------------------------------------------------------------        


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-15 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Coordinates: x1=62, y1=36, x2=198, y2=146


True

In [65]:
!rm -rf /kaggle/working/exp

### **Checking if the image is already greyscaled**

In [66]:
import cv2
import numpy as np

def is_grayscale(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Check if the image is single channel (grayscale)
    if len(image.shape) == 2:  # Grayscale image will have shape (height, width)
        return True
    
    # If the image has 3 channels, check if all channels are the same
    if len(image.shape) == 3 and image.shape[2] == 3:  # RGB image will have shape (height, width, 3)
        # Split the image into its three channels
        b, g, r = cv2.split(image)
        # Check if all channels are equal (grayscale has equal R, G, B values)
        if np.array_equal(b, g) and np.array_equal(g, r):
            return True
    
    return False

# Example usage
image_path = '/kaggle/working/results_person_removed.jpg'
if is_grayscale(image_path):
    print("Image is grayscale")
    grayscale_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Increase the brightness and contrast of the image (if needed)
    alpha = 1  # Contrast control (1.0-3.0)
    beta = 0    # Brightness control (0-100)

    # Adjust brightness and contrast
    brightened_image = cv2.convertScaleAbs(grayscale_image, alpha=alpha, beta=beta)

    # Apply a threshold to turn gray background to white
    _, thresholded_image = cv2.threshold(brightened_image, 200, 255, cv2.THRESH_BINARY)

    # Convert the resulting image to RGB
    rgb_image = cv2.cvtColor(thresholded_image, cv2.COLOR_GRAY2RGB)

    # Save or display the RGB image
    cv2.imwrite('/kaggle/working/results_image.jpg', rgb_image)
else:
    print("The image is not grayscale.")
    rgb_image = cv2.imread(image_path)
    cv2.imwrite('/kaggle/working/results_image.jpg', rgb_image)


The image is not grayscale.


### **Creating 2 columns basis of space**

In [67]:
# import numpy as np
# import cv2

# # Assuming you've already loaded the image
# image = cv2.imread('/kaggle/working/results_image.jpg')

# def count_distinct_colors(image, color_space='RGB'):
#     # Convert image to the specified color space
#     if color_space == 'HSV':
#         converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     elif color_space == 'Lab':
#         converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
#     else:
#         converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
#     # Reshape the image to a 2D array of pixels
#     pixels = converted_image.reshape(-1, 3)
    
#     # Convert to a set of tuples (each tuple represents a unique color)
#     unique_colors = set(map(tuple, pixels))
    
#     return len(unique_colors)

# # Count distinct colors in different color spaces
# rgb_count = count_distinct_colors(image, 'RGB')
# hsv_count = count_distinct_colors(image, 'HSV')
# lab_count = count_distinct_colors(image, 'Lab')

# print(f"Number of distinct colors in RGB space: {rgb_count}")
# print(f"Number of distinct colors in HSV space: {hsv_count}")
# print(f"Number of distinct colors in Lab space: {lab_count}")

In [68]:
# import cv2
# import numpy as np

# # Load the image
# r_image = cv2.imread("/kaggle/working/results_person_removed.jpg")
# fixed_size = (1250,1563)
# image = cv2.resize(r_image, fixed_size)
# cv2.imwrite("/kaggle/working/2nd_resized_image.jpg",image)
# image = cv2.imread("/kaggle/working/2nd_resized_image.jpg")
# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply binary thresholding to get a binary image
# _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

# # Find the contours of the text regions
# contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# # Create a mask for the text regions
# mask = np.zeros_like(binary)

# # Draw filled contours on the mask (text regions become white)
# cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)

# # Invert the mask to get the non-text areas (non-text areas become white)
# mask_inv = cv2.bitwise_not(mask)
# # Find the contours of the text regions
# cv2.drawContours(mask_inv, contours, 1, 255, thickness=cv2.FILLED)

# unique_background_colors = 1

# for contour in contours:
# #     print(contour)
#     x, y, w, h = cv2.boundingRect(contour)
# #     cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 3)

#     if h > 500 and w > 150:
#         unique_background_colors = 2
       
# if unique_background_colors == 2:  # Two background colors detected
#     print("Two background colors detected. Keeping the image unchanged.")
# else:
# # Parameters
#     height, width = mask_inv.shape
#     line_thickness = 2
#     required_non_text_percentage = 0.98
#     # 98% non-text space requirement(0.9780 - 0.98)
#     required_consecutive_columns =  40
#     #Number of consecutive columns that need to meet the non-text requirement(40-25)
#     non_text_threshold = int(height * required_non_text_percentage)
#     initial_skip_percentage = 0.2      # Skip the first 20% of the x-axis#@@@@@@@@@@@@@@@@@

#     # Calculate the starting point after skipping the initial part
#     start_x = int(width * initial_skip_percentage)

#     # Initialize flag and counters
#     consecutive_non_text_columns = 0
#     best_x = -1

#     # Scan across the x-axis starting after the initial skip
#     for x in range(start_x, width):
#         # Calculate the number of non-text pixels along the y-axis at this x position
#         non_text_pixels = np.sum(mask_inv[:, x]) / 255

#         # Check if this position meets the non-text requirement
#         if non_text_pixels >= non_text_threshold:
#             consecutive_non_text_columns += 1
#             # If we have enough consecutive columns, we can consider this position
#             if consecutive_non_text_columns >= required_consecutive_columns:
#                 best_x = x - (consecutive_non_text_columns // 2)  # Center the line
#                 break
#         else:
#             # Reset the counter if the requirement is not met
#             consecutive_non_text_columns = 0

#     # Draw the vertical line at the found position if it exists
#     if best_x != -1:
#          # Define the background color (e.g., light blue)
#         background_color = [255, 100, 255]  # BGR format for light blue

#         # Create a colored rectangle for the background change
#         background = np.full_like(image, background_color)# BGR format for light blue

#          # Convert the mask to a three-channel image to match the color image
#         mask_3channel = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

#         # Apply the background color only to the area before the line, ignoring text
#         image[:, :best_x] = np.where(mask_3channel[:, :best_x] == [0, 0, 0], background[:, :best_x], image[:, :best_x])

#         cv2.line(image, (best_x, 0), (best_x, height), (0,0,0), thickness=line_thickness)
#         print(f"Line drawn at x={best_x}.")
#     else:
#         print("No suitable vertical space found for drawing the line.")

# # Save or display the result
# cv2.imwrite('/kaggle/working/output_image_with_line.png', image)


In [69]:
import cv2
import numpy as np
# Load the image
r_image = cv2.imread("/kaggle/working/results_image.jpg")
fixed_size = (1250,1563)
image = cv2.resize(r_image, fixed_size)
cv2.imwrite("/kaggle/working/2nd_resized_image.jpg",image)
image = cv2.imread("/kaggle/working/2nd_resized_image.jpg")


# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
a = 0
for i in range(70,150,5):
    # Apply binary thresholding to get a binary image
    _, binary = cv2.threshold(gray, i, 255, cv2.THRESH_BINARY_INV)

    # Find the contours of the text regions
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Create a mask for the text regions
    mask = np.zeros_like(binary)

    # Draw filled contours on the mask (text regions become white)
    cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)

    # Invert the mask to get the non-text areas (non-text areas become white)
    mask_inv = cv2.bitwise_not(mask)
    # Get the unique pixel values
    # unique_colors = np.unique(gray)
    # # print(unique_colors)
    # prev_unique_color = unique_colors_count = 0
    # for i in unique_colors:
    #     if i - prev_unique_color > 55:
    #         prev_unique_color = i
    #         print(i)
    #         unique_colors_count = unique_colors_count+1
    # print(unique_colors_count)
    #Find the contours of the text regions
    # Convert the image to HSV color space
    converted_image = cv2.cvtColor(r_image, cv2.COLOR_BGR2RGB)

    # Reshape the image to a 2D array of pixels
    pixels = converted_image.reshape(-1, 3)

    # Convert to a set of unique colors
    unique_colors = set(map(tuple, pixels))

    # Convert the set back to an array and sort it by the first channel (Hue)
    unique_colors_array = np.array(list(unique_colors))
    unique_colors_array = unique_colors_array[unique_colors_array[:, 0].argsort()]

    # Initialize the colors list with the first unique color's first channel value
    colors = [unique_colors_array[0]]

    # Iterate through the unique colors (sorted)
    for i in range(1, len(unique_colors_array)):
        # Only add the color if the difference with the last color is greater than 55
        if abs(unique_colors_array[i][0] - colors[-1][0]) > 55:
            colors.append(unique_colors_array[i])

    # Print the selected colors
    print(colors)
    if len(colors)> 2:
        unique_background_colors = 1
        # Draw filled contours on the mask (text regions become white)
    #     cv2.drawContours(mask_inv, contours, 1, 255, thickness=cv2.FILLED)
    else:
        unique_background_colors = 0 
    cv2.drawContours(mask_inv, contours, 0, 255, thickness=cv2.FILLED)

# unique_background_colors = 0 # 0/1
# cv2.drawContours(mask_inv, contours, -1, 255, thickness=cv2.FILLED)

    prev_contour = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if h > 1000 and w > 80 and w < 1200 :#h =800
#             print(h)
            a = 1
#             cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 3)
            unique_background_colors = unique_background_colors + 1
        
    if a==1:
        break
print("No. of background parts with different colour found: ",unique_background_colors)

     
if unique_background_colors > 1:  # Two background colors detected
    print("Two background colors detected. Keeping the image unchanged.")
else:

# Parameters
    height, width = mask_inv.shape
    line_thickness = 2
    required_non_text_percentage = 0.1
    # 98% non-text space requirement(0.9780 - 0.98)
    required_consecutive_columns =  40
    #Number of consecutive columns that need to meet the non-text requirement(40-25)
   
    initial_skip_percentage = 0.2      # Skip the first 20% of the x-axis#@@@@@@@@@@@@@@@@@
    f = 0
    c = 0 
    best_x = -1
    for required_consecutive_columns in range(65,25,-1):
        y = required_consecutive_columns
        for required_non_text_percentage1 in range (1000,949,-1):####1000000- to be precise
            required_non_text_percentage = required_non_text_percentage1 / 1000.00
            x = required_non_text_percentage
            if required_non_text_percentage > 0.99 and c==0:
                c = 1
                for required_consecutive_columns in range(200,10,-1):
                        non_text_threshold = int(height * required_non_text_percentage)
                        # Calculate the starting point after skipping the initial part
                        start_x = int(width * initial_skip_percentage)

                        # Initialize flag and counters
                        consecutive_non_text_columns = 0

                        # Scan across the x-axis starting after the initial skip
                        for x in range(start_x, width):
                            # Calculate the number of non-text pixels along the y-axis at this x position
                            non_text_pixels = np.sum(mask_inv[:, x]) / 255

                            # Check if this position meets the non-text requirement
                            if non_text_pixels >= non_text_threshold:
                                consecutive_non_text_columns += 1
                                # If we have enough consecutive columns, we can consider this position
                                if consecutive_non_text_columns >= required_consecutive_columns:
                                    best_x = x - (consecutive_non_text_columns // 2)  
                                    break
                            else:
                                # Reset the counter if the requirement is not met
            #                     print(x)
                                consecutive_non_text_columns = 0
                        if best_x < 900 and best_x!= -1:
                            f = 1
                            break
        
#             if f == 1:
#                 # Define the background color (e.g., light blue)
#                 background_color = [255, 100, 255]  # BGR format for light blue

#                 # Create a colored rectangle for the background change
#                 background = np.full_like(image, background_color)# BGR format for light blue

#                 # Convert the mask to a three-channel image to match the color image
#                 mask_3channel = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
#                 # Apply the background color only to the area before the line, ignoring text
#                 image[:, :best_x] = np.where(mask_3channel[:, :best_x] == [0, 0, 0], background[:, :best_x], image[:, :best_x])
#                 cv2.line(image, (best_x, 0), (best_x, height), (0,0,0), thickness=line_thickness)
#                 required_non_text_percentage = required_non_text_percentage*100
#                 print(f"Line drawn at x={best_x} with {required_consecutive_columns} consecutive columns with {required_non_text_percentage}% of non-text")
#                 break
#                         if best_x == -1 or f==0:
#                             print("No suitable vertical space found for drawing the line.")       
                        
            else: 
                required_non_text_percentage = x
                required_consecutive_columns = y
                non_text_threshold = int(height * required_non_text_percentage)
                # Calculate the starting point after skipping the initial part
                start_x = int(width * initial_skip_percentage)
                # Initialize flag and counters
                consecutive_non_text_columns = 0

                # Scan across the x-axis starting after the initial skip
                for x in range(start_x, width):
                    # Calculate the number of non-text pixels along the y-axis at this x position
                    non_text_pixels = np.sum(mask_inv[:, x]) / 255

                    # Check if this position meets the non-text requirement
                    if non_text_pixels >= non_text_threshold:
                        
                        consecutive_non_text_columns += 1
                        # If we have enough consecutive columns, we can consider this position
                        if consecutive_non_text_columns >= required_consecutive_columns:
#                             print(non_text_pixels)
                            best_x = x - (consecutive_non_text_columns // 2)
                            break
                    else:
                        # Reset the counter if the requirement is not met
#                         print(x)
                        consecutive_non_text_columns = 0
                        
                    if best_x < 1000 and best_x != -1:
#                         print(best_x)
                        f = 1
                        break
        
                
            if f == 1:
                # Define the background color (e.g., light blue)
                background_color = [255, 100, 255]  # BGR format for light blue

                # Create a colored rectangle for the background change
                background = np.full_like(image, background_color)# BGR format for light blue

                # Convert the mask to a three-channel image to match the color image
                mask_3channel = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
                # Apply the background color only to the area before the line, ignoring text
                image[:, :best_x] = np.where(mask_3channel[:, :best_x] == [0, 0, 0], background[:, :best_x], image[:, :best_x])
                cv2.line(image, (best_x, 0), (best_x, height), (0,0,0), thickness=line_thickness)
                required_non_text_percentage = required_non_text_percentage*100
                print(f"Line drawn at x={best_x} with {required_consecutive_columns} consecutive columns with {required_non_text_percentage}% of non-text")
                break
        if f == 1:
            break
    if best_x == -1 or f==0:#
#         print(best_x)
        print("No suitable vertical space found for drawing the line.")
c == f
# Save or display the result
cv2.imwrite('/kaggle/working/output_image_with_line.png',image)

[array([64, 64, 64], dtype=uint8), array([120, 120, 120], dtype=uint8), array([176, 176, 176], dtype=uint8), array([232, 232, 232], dtype=uint8)]
[array([64, 64, 64], dtype=uint8), array([120, 120, 120], dtype=uint8), array([176, 176, 176], dtype=uint8), array([232, 232, 232], dtype=uint8)]
[array([64, 64, 64], dtype=uint8), array([120, 120, 120], dtype=uint8), array([176, 176, 176], dtype=uint8), array([232, 232, 232], dtype=uint8)]
[array([64, 64, 64], dtype=uint8), array([120, 120, 120], dtype=uint8), array([176, 176, 176], dtype=uint8), array([232, 232, 232], dtype=uint8)]
[array([64, 64, 64], dtype=uint8), array([120, 120, 120], dtype=uint8), array([176, 176, 176], dtype=uint8), array([232, 232, 232], dtype=uint8)]
[array([64, 64, 64], dtype=uint8), array([120, 120, 120], dtype=uint8), array([176, 176, 176], dtype=uint8), array([232, 232, 232], dtype=uint8)]
[array([64, 64, 64], dtype=uint8), array([120, 120, 120], dtype=uint8), array([176, 176, 176], dtype=uint8), array([232, 232

True

### **Greyscaling of Images**

In [70]:
import pytesseract
import cv2
image = cv2.imread("/kaggle/working/output_image_with_line.png")

# # creating a image object
# im1 = cv2.imread(r"/content/0-0_jpg.rf.91a1b5ecfa392c10b1986ad8070ffb2b11111.jpg")

# # copying image to another image object
# im2 = im1.copy()

# # shows the copied image
# cv2.imwrite("/content/combined_gridded.png",im2)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("/kaggle/working/temp_gray.png",gray)

True

### **Threshold level Merging**

In [71]:
_, thresh_black = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
_, thresh_white = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
# _, binary_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Combine both thresholds

thresh1 = cv2.bitwise_or(thresh_black, thresh_white)

# thresh2 = cv2.threshold(binary_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
# combined_thresh = cv2.bitwise_or(thresh1, thresh2)

cv2.imwrite("/kaggle/working/temp_thres_white.png",thresh1)
# cv2.imwrite("/kaggle/working/temp_thres_black.png",thresh2)


True

In [74]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
o1 = original_image = cv2.imread('/kaggle/working/2nd_resized_image.jpg')
# Load the image
image = cv2.imread('/kaggle/working/temp_thres_white.png', cv2.IMREAD_GRAYSCALE)

# Threshold the image to get binary image (assuming black is 0 and white is 255)
_, binary_image = cv2.threshold(image, 1, 255, cv2.THRESH_BINARY_INV)
cv2.imwrite("/kaggle/working/binary.jpg",binary_image)

# Find contours in the binary image
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# Initialize a variable to store the minimum x-coordinate of the green line
min_x = max_x = min_x_w = original_image.shape[1]  # Start with the width of the image
# # Convert grayscale image to BGR to draw colored rectangles
# image_bgr = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
f = max_y = prev_x = 0
# prev_y = prev_x = prev_h = f = 0
# Draw red rectangles around the contours
# if best_x == -1 or 
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    

    if h > 500:
#         min_x_w =  min(min_x, x+w)
#         min_x = min(min_x, x) # Update min_x with the smallest x-coordinate
#         max_x = max(max_x,w)
#         print(contour)
#         max_y = 0# Initialize max_y with the top of the bounding rectangle
        for point in contour:
            # print(point)
            current_y = point[0][1]
            if current_y >= max_y and point[0][1]< 1100:
                # print(point[0][0])
                max_y = current_y
                prev_x = point[0][0]
#                 w = point[0][0]##########
            else:
                try:
                    # Attempt to access the variable
                    print(best_x)
                except NameError:
                    if point[0][0] < 1100:
                        w1 = prev_x
                    # Handle the case where the variable is not defined
                else:
                    if point[0][0] < 1100 or best_x < 1100:
                        if best_x != -1 or c!=0:
                            w1 = best_x
                        else:
                            w1 = point[0][0]
                        # w = prev_x###############
                # print(w)
                    f = 1
                    break
#                    # We've found the highest point, so we can stop searching  
    if f==1:
            break
                
min_x = min(min_x, x) # Update min_x with the smallest x-coordinate
min_x_w =  min(min_x_w, x+w,w1)
# print(w1)
# print(min_x_w)
max_x = max(max_x,w)
cv2.rectangle(original_image, (x, y), (w, max_y), (0, 255, 0), 3)
            
print(min_x_w)
print(min_x)
print(max_x)
# print(best_x)
# Save or display the resulting image
if min_x > 500 or min_x_w > min_x:
    if min_x < 20:
    
        min_x = min_x_w
    
    cropped_image_left = original_image[:,:min_x]
    cropped_image_right = original_image[:,min_x:max_x]
else:
    cropped_image_left = original_image[:, :min_x_w]
    cropped_image_right = original_image[:,min_x_w:]
# # Save or display the cropped image
# plt.imshow(cropped_image_left)
# # plt.imshow(cropped_image_right)
if cropped_image_left is not None and cropped_image_left.any():
    cv2.imwrite('/kaggle/working/cropped_image_left.png', cropped_image_left)
    if cropped_image_right is not None and cropped_image_right.any():
        cv2.imwrite('/kaggle/working/cropped_image_right.png',cropped_image_right)
    else:
        print("Single column so no right part found!!")
else:
    print("Single column")
    cv2.imwrite('/kaggle/working/cropped_image_left.png',o1)
# plt.axis('off')  # Hide the axis if you just want to display the image
# plt.show()

450
449
0
1250


In [137]:
# !rm -rf /kaggle/working/resume_text137.txt

In [73]:
from gradio_client import Client, handle_file


import cv2
import os

left_text=right_text=''

left_image = '/kaggle/working/cropped_image_left.png'
client = Client("artificialguybr/Surya-OCR")
result = client.predict(
		image=handle_file(left_image),
		langs="en",
		api_name="/ocr_workflow"
    )
# print(result)
result1= list(result)

left_text = result1[-1]
# print(left_text)

right_image = '/kaggle/working/cropped_image_right.png'
if os.path.exists(right_image):
    client = Client("artificialguybr/Surya-OCR")
    result_r = client.predict(
		image=handle_file(right_image),
		langs="en",
		api_name="/ocr_workflow"
    )
    # print(result)
    result2= list(result_r)
    right_text = result2[-1]
    # print(result1[-1])
    
with open("/kaggle/working/Resume129.txt", "a") as f: ################
    fr = ''
    fr = fr + str("First column: " )+"\n" 
    fr = fr + left_text + '\n'
#     for i in range(len(left_text)): 
#         fr = fr + str(left_text[i]) + "\n" 
#         # f.write("First column: ")

    fr1 = ''
    fr1 = fr1 + str("Second column: " )+"\n"
    fr1 =fr1+ right_text + '\n' + "==================================================================" + "\n"
# for i in range(len(right_text)):
#     # You were trying to access filtered_non_green_text[0][i] which is incorrect as filtered_non_green_text is a list of strings
#     fr1 = fr1 + str(right_text[i]) + "\n"
    fr = fr + fr1
    f.write(fr)
white_page = cv2.imread("/kaggle/input/white-page/Screenshot 2024-09-21 043703.png")
cv2.imwrite('/kaggle/working/cropped_image_right.png',white_page)
cv2.imwrite('/kaggle/working/cropped_image_left.png',white_page)

Loaded as API: https://artificialguybr-surya-ocr.hf.space ✔
Loaded as API: https://artificialguybr-surya-ocr.hf.space ✔


True

### **Paddle OCR extraction**

In [18]:
from paddleocr import PaddleOCR
import cv2
import os
ocr = PaddleOCR(use_angle_cls=True,lang='en')

left_text=right_text=''

# fixed_size = (1250,1563)
# l_image = cv2.imread("/kaggle/working/cropped_image_left.png")
# image = cv2.resize(l_image, fixed_size)
# cv2.imwrite("/kaggle/working/2nd_resized_image.jpg",image)
# r_image = cv2.imread("/kaggle/working/cropped_image_right.png")
# image = cv2.resize(r_image, fixed_size)
# cv2.imwrite("/kaggle/working/2nd_resized_image.jpg",image)

left_image = '/kaggle/working/cropped_image_left.png'
result = ocr.ocr(left_image)
for line in result:
    if line is not None:
        for word_info in line:
            text = word_info[1][0]
            if text is not None:
    #             print(text)
                left_text = left_text + text + '\n'
# print(left_text)

right_image = '/kaggle/working/cropped_image_right.png'
if os.path.exists(right_image):
    result = ocr.ocr(right_image, cls=True)
    for line in result:
        if line is not None:
            for word_info in line:
                text = word_info[1][0]
                if text is not None:
                    right_text = right_text + text + '\n'
with open("/kaggle/working/cv_new.txt", "a") as f: ################
    fr = ''
    fr = fr + str("First column: " )+"\n" 
    fr = fr + left_text + '\n'
#     for i in range(len(left_text)): 
#         fr = fr + str(left_text[i]) + "\n" 
#         # f.write("First column: ")

    fr1 = ''
    fr1 = fr1 + str("Second column: " )+"\n"
    fr1 =fr1+ right_text + '\n' + "==================================================================" + "\n"
# for i in range(len(right_text)):
#     # You were trying to access filtered_non_green_text[0][i] which is incorrect as filtered_non_green_text is a list of strings
#     fr1 = fr1 + str(right_text[i]) + "\n"
    fr = fr + fr1
    f.write(fr)
white_page = cv2.imread("/kaggle/input/white-page/Screenshot 2024-09-21 043703.png")
cv2.imwrite('/kaggle/working/cropped_image_right.png',white_page)
cv2.imwrite('/kaggle/working/cropped_image_left.png',white_page)

PLEASE USE OMP_NUM_THREADS WISELY.


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:19<00:00, 202.81it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:18<00:00, 527.01it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:15<00:00, 134.10it/s]

[2024/12/08 15:07:10] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

[2024/12/08 15:07:11] ppocr DEBUG: dt_boxes num : 45, elapsed : 0.17926549911499023
[2024/12/08 15:07:11] ppocr DEBUG: cls num  : 45, elapsed : 0.1522059440612793
[2024/12/08 15:07:17] ppocr DEBUG: rec_res num  : 45, elapsed : 5.714731931686401
[2024/12/08 15:07:18] ppocr DEBUG: dt_boxes num : 51, elapsed : 0.3309452533721924
[2024/12/08 15:07:18] ppocr DEBUG: cls num  : 51, elapsed : 0.10141253471374512
[2024/12/08 15:07:31] ppocr DEBUG: rec_res num  : 51, elapsed : 12.962459564208984


True

In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# # Load the image
# image = cv2.imread('/content/6db3beabacf07d2ef8dfe3602c8a7914_png.rf.7f123796ce4d47c264b21327b04b9712_page-0001.jpg')
# # Convert to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply binary thresholding to invert the image (text becomes white, background becomes black)
# _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# # Find contours
# contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Sort contours based on the x-coordinate (to identify the columns)
# contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])
# cv2.drawContours(image, contours, -1, (0, 255, 0), 2)
# plt.imshow(image)
# plt.show
# # Assume there are two main contours (columns), find their bounding rectangles
# left_col = cv2.boundingRect(contours[0])
# right_col = cv2.boundingRect(contours[-1])

# # Calculate the middle of the space between the two columns
# print("L0",left_col[0],"L2",left_col[2])
# print("R0",right_col[0],"R2",right_col[2])
# ############################Solve below line..
# middle_x = (max(right_col[0],right_col[2])-((left_col[2]//(right_col[2])) * (1+left_col[0]) )-min(left_col[0],left_col[2]))  // 2
# # #- ((max(left_col[0],left_col[2]) + 2 * min(left_col[0],left_col[2]))
# # middle_x = (  right_col[0] + left_col[0] + left_col[2])  // 2
# # Draw the vertical line at the middle of the space
# cv2.line(image, (middle_x, 0), (middle_x, image.shape[0]), (0, 0, 0), 2)

# # Save or display the result
# cv2.imwrite('document_with_divider.png', image)

# # #-----------------------------------------------------
# # # Calculate the middle x-coordinate between the two columns
# # middle_x = (left_col[0] + left_col[2] + right_col[0]) // 2

# # # Draw the vertical line at the middle of the space from top to bottom
# # cv2.line(image, (middle_x, 0), (middle_x, image.shape[0]), (0, 0, 0), 2)

# # # Save or display the result
# # cv2.imwrite('/mnt/data/document_with_divider.png', image)

# # # or
# # # plt.imshow('Image with Divider', image)
# # # plt.show()
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()




In [ ]:
# import cv2
# import numpy as np

# # Load the image
# image = cv2.imread('/content/6db3beabacf07d2ef8dfe3602c8a7914_png.rf.7f123796ce4d47c264b21327b04b9712_page-0001.jpg')

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply binary thresholding
# _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# # Find the contours
# contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Create a mask for drawing
# mask = np.zeros_like(binary)

# # Draw the contours on the mask
# cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)

# # Invert the mask to get the spaces
# mask_inv = cv2.bitwise_not(mask)

# # Sum up the columns of the mask_inv to get the column densities
# column_densities = np.sum(mask_inv, axis=0)

# # Find the index with the lowest density which should be the vertical space between columns
# min_index = np.argmin(column_densities)

# # Draw a vertical line at the min_index position
# line_thickness = 2
# cv2.line(image, (min_index, 0), (min_index, image.shape[0]), (0, 0, 255), thickness=line_thickness)

# # Save or display the result
# cv2.imwrite('output_image_with_line.png', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# import cv2
# import numpy as np

# # Load the image
# image = cv2.imread('/content/5f5a93dc-4946-425b-8d44-17fcf8ba3471-1_jpg.rf.440f1b75afa3cc5f782eeb5aeb0956d4_page-0001.jpg')

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply binary thresholding to get a binary image
# _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# # Find the contours of the text regions
# contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Create a mask for the text regions
# mask = np.zeros_like(binary)

# # Draw filled contours on the mask (text regions become white)
# cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)

# # Invert the mask to get the non-text areas (non-text areas become white)
# mask_inv = cv2.bitwise_not(mask)

# # Start searching for the vertical line position
# height, width = mask_inv.shape
# best_x = -1
# max_non_text_percentage = 0.8  # 80% non-text space requirement
# flag = 0

# # Scan across the x-axis to find the best position
# for x in range(width):
#     # Calculate the percentage of non-text pixels along the y-axis at this x position
#     non_text_pixels = np.sum(mask_inv[:, x]) / 255
#     non_text_percentage = non_text_pixels / height

#     # Check if this position meets the 80% non-text requirement
#     if non_text_percentage >= max_non_text_percentage:
#       if x == 0:
#         continue
#       else:
#         best_x = x
#         break

# # Draw the vertical line at the found position if it exists
# if best_x != -1:
#     line_thickness = 2
#     cv2.line(image, (best_x, 0), (best_x, height), (0, 0, 255), thickness=line_thickness)
#     print(f"Line drawn at x={best_x} where non-text percentage is {non_text_percentage*100:.2f}%")
# else:
#     print("No suitable vertical space found for drawing the line.")

# # Save or display the result
# cv2.imwrite('/content/output_image_with_line.png', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# import cv2
# import numpy as np

# # Function to find the largest vertical gap in a column
# def find_largest_vertical_gap(binary_column):
#     max_gap = 0
#     max_start = 0
#     gap_start = -1
#     in_gap = False

#     for i, pixel in enumerate(binary_column):
#         if pixel == 0:  # If pixel is black (object)
#             if in_gap:
#                 gap_size = i - gap_start
#                 if gap_size > max_gap:
#                     max_gap = gap_size
#                     max_start = gap_start
#                 in_gap = False
#         else:  # If pixel is white (background)
#             if not in_gap:
#                 gap_start = i
#                 in_gap = True

#     # Check the last gap
#     if in_gap:
#         gap_size = len(binary_column) - gap_start
#         if gap_size > max_gap:
#             max_gap = gap_size
#             max_start = gap_start

#     return max_start, max_start + max_gap

# # Load the image
# image = cv2.imread('/content/6db3beabacf07d2ef8dfe3602c8a7914_png.rf.7f123796ce4d47c264b21327b04b9712_page-0001.jpg')
# height, width = image.shape[:2]

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply a threshold to get a binary image
# _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# # Initialize variables to store the column with the largest gap
# largest_gap_column = 0
# largest_gap_start = 0
# largest_gap_end = 0
# largest_gap_size = 0

# # Iterate over each column to find the one with the largest vertical gap
# for col in range(width):
#     start, end = find_largest_vertical_gap(binary[:, col])
#     gap_size = end - start
#     if gap_size > largest_gap_size:
#         largest_gap_size = gap_size
#         largest_gap_start = start
#         largest_gap_end = end
#         largest_gap_column = col

# # Draw a line at the center of the largest vertical gap
# if largest_gap_size > 0:
#     center_y = (largest_gap_start + largest_gap_end) // 2
#     cv2.line(image, (largest_gap_column - 10, center_y), (largest_gap_column + 10, center_y), (0, 255, 0), 2)

# # Display the result
# cv2.imwrite('/content/document_with_divider.png', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# import cv2
# image = cv2.imread("/content/5efae9e3b8c75913e210bf72_example-real-estate-agent-resume_png.rf.6c223485926112f23ca452c0f9c76e2b_page-0001.jpg")
# image.shape

image1 = cv2.imread('/kaggle/working/original_with_green_boxes1.png')
image2 = cv2.imread('/kaggle/working/original_with_green_boxes2.png')
final_img = cv2.bitwise_or(image1, image2)
cv2.imwrite("/kaggle/working/final_img_with_green_boxes.png",final_img)

In [ ]:
# import cv2
# import numpy as np
# from paddleocr import PaddleOCR
# import matplotlib.pyplot as plt

# # Load the image
# image_path = '/kaggle/working/original_with_green_boxes1.png'
# image = cv2.imread(image_path)

# # Convert image to HSV color space for color detection
# hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# # Define the range for green color in HSV space
# lower_green = np.array([35, 100, 100])  # Lower bound of green color
# upper_green = np.array([85, 255, 255])  # Upper bound of green color

# # Create a mask for green color
# mask = cv2.inRange(hsv_image, lower_green, upper_green)

# # Find contours of the green boxes
# contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Create a mask to differentiate between green and non-green areas
# non_green_mask = cv2.bitwise_not(mask)
# # Initialize PaddleOCR
# ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Enable angle classification and set language to English

# def get_text_from_mask(image, mask):
#     """Extract text from the image using a given mask."""
#     masked_image = cv2.bitwise_and(image, image, mask=mask)
#     results = ocr.ocr(masked_image, cls=True)
#     return results

# # Process green boxes
# roi_texts = []
# for contour in contours:
#     x, y, w, h = cv2.boundingRect(contour)
#     roi_mask = np.zeros_like(mask)
#     cv2.drawContours(roi_mask, [contour], -1, 255, thickness=cv2.FILLED)
#     text = get_text_from_mask(image, roi_mask)
#     roi_texts.append(text)

# # Process non-green areas
# non_green_text = get_text_from_mask(image, non_green_mask)

# # Function to filter text
# def filter_text_only(results):
#     text_only = []
#     for res in results:
#         if res:
#             for line in res:
#               ###############################################Problem else all done
#                 text_only.append(line[-1][0])  # Extract the text from the result
#         return text_only
#     # for res in results:
#     #     text = res[1]
#     #     # Use regex to check if the string contains any numerical characters
#     #     if not re.search(r'\d', text):
#     #         text_only.append(text)
#     # return text_only

# # Apply the filter to each ROI's OCR results
# filtered_roi_texts = [filter_text_only(roi_text) for roi_text in roi_texts]
# filtered_non_green_text = filter_text_only(non_green_text)

# # Write all the text to the file within a single 'with' block
# with open("/kaggle/working/resume_text2.txt", "a") as f:
#     fr = ''
#     fr = fr + str("First column: " )+"\n"
#     for i in range(len(filtered_roi_texts[0])):
#         fr = fr + str(filtered_roi_texts[0][i]) + "\n"
#     # f.write("First column: ")


#     fr1 = ''
#     fr1 = fr1 + str("Second column: " )+"\n"
#   # Added a newline character here for better formatting
#     for i in range(len(filtered_non_green_text)):
#         # You were trying to access filtered_non_green_text[0][i] which is incorrect as filtered_non_green_text is a list of strings
#         fr1 = fr1 + str(filtered_non_green_text[i]) + "\n"
#     fr = fr + fr1
#     f.write(fr)

In [ ]:
# import cv2
# import numpy as np

# # Load the image
# original_image = cv2.imread('/kaggle/input/r123456/5efae9e3b8c75913e210bf72_example-real-estate-agent-resume_png.rf.6c223485926112f23ca452c0f9c76e2b_page-0001.jpg')
# image = cv2.imread('/kaggle/working/temp_thres_black.png', cv2.IMREAD_GRAYSCALE)

# # Threshold the image to get binary image (assuming black is 0 and white is 255)
# _, binary_image = cv2.threshold(image, 1, 255, cv2.THRESH_BINARY_INV)

# # Find contours in the binary image
# contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Convert grayscale image to BGR to draw colored rectangles
# image_bgr = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

# # Draw red rectangles around the contours
# for contour in contours:
#     x, y, w, h = cv2.boundingRect(contour)
#     if h > 100:
#       cv2.rectangle(original_image, (x, y), (x+w, y+h), (0,255, 0), 3)


# # Save or display the resulting image
# cv2.imwrite('/kaggle/working/original_with_green_boxes2.png', original_image)


In [ ]:
# import cv2
# import easyocr
# import numpy as np
# import re

# # Load the image
# image_path = '/content/final_img_with_green_boxes.png'
# image = cv2.imread(image_path)

# # Convert image to HSV color space for color detection
# hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# # Define the range for green color in HSV space
# lower_green = np.array([35, 100, 100])  # Lower bound of green color
# upper_green = np.array([85, 255, 255])  # Upper bound of green color

# # Create a mask for green color
# mask = cv2.inRange(hsv_image, lower_green, upper_green)

# # Find contours of the green boxes
# contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Create a mask to differentiate between green and non-green areas
# non_green_mask = cv2.bitwise_not(mask)

# # Initialize EasyOCR Reader
# reader = easyocr.Reader(['en'])

# def get_text_from_mask(image, mask):
#     """Extract text from the image using a given mask."""
#     masked_image = cv2.bitwise_and(image, image, mask=mask)
#     results = reader.readtext(masked_image)
#     return results

# # Process green boxes
# roi_texts = []
# for contour in contours:
#     x, y, w, h = cv2.boundingRect(contour)
#     roi_mask = np.zeros_like(mask)
#     cv2.drawContours(roi_mask, [contour], -1, 255, thickness=cv2.FILLED)
#     text = get_text_from_mask(image, roi_mask)
#     roi_texts.append(text)

# # Process non-green areas
# non_green_text = get_text_from_mask(image, non_green_mask)

# # Function to filter out numerical values
# def filter_text_only(results):
#     text_only = []
#     for res in results:
#         text = res[1]

#         text_only.append(text)
#     return text_only

# # Apply the filter to each ROI's OCR results
# filtered_roi_texts = [filter_text_only(roi_text) for roi_text in roi_texts]
# filtered_non_green_text = filter_text_only(non_green_text)

# # Print the filtered text
# # print("Filtered Text from Green Boxes:", filtered_roi_texts)
# # print("Filtered Text from Non-Green Areas:", filtered_non_green_text)
# fr = ''
# for i in range(len(filtered_roi_texts[0])):
#   fr = fr + str(filtered_roi_texts[0][i]) + " "
# f = open("/content/resume_text.txt", "a")
# f.write(fr)
# f.close()
# print(filtered_non_green_text)


In [ ]:
  # import cv2
# import easyocr

# # Load the image
# image = cv2.imread('/content/final_img_with_green_boxes.png')

# # Initialize the EasyOCR reader
# reader = easyocr.Reader(['en'])

# # Perform OCR to detect text
# results = reader.readtext(image)
# # prev_h = 0
# # prev_bottom_right = prev_top_left = (0,0)
# # Iterate through the detected text and extract dimensions
# for (bbox, text, prob) in results:
#     # bbox: bounding box coordinates [(top_left), (top_right), (bottom_right), (bottom_left)]
#     # text: the recognized text
#     # prob: the probability of the recognition

#     # Unpack bounding box coordinates
#     # (top_left, top_right, bottom_right, bottom_left) = bbox

#     # # Convert the coordinates to integers
#     # top_left = tuple(map(int, top_left))
#     # bottom_right = tuple(map(int, bottom_right))

#     # # Calculate dimensions
#     # w = bottom_right[0] - top_left[0]
#     # h = bottom_right[1] - top_left[1]
#     print(f"{text}")
#     # print("top_left:",top_left,"bottom_right:",bottom_right,"width:",w,"Height:",h)
#     # cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)


# # Display the image with bounding boxes
# # cv2.imwrite('/content/dimension1.jpg',image)


# #
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# import cv2
# import numpy as np
# from matplotlib import pyplot as plt

# # Load the original image and the image with green boxes
# original_image = cv2.imread('/content/0-0_jpg.rf.91a1b5ecfa392c10b1986ad8070ffb2b11111.jpg')  # Path to the original image
# image_with_boxes = cv2.imread('/content/output_image.jpg')  # Path to the image with green boxes

# # Convert the image with green boxes to HSV (Hue, Saturation, Value) color space
# hsv = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2HSV)

# # Define the range for green color in HSV space
# lower_green = np.array([40, 40, 40])  # Lower bound of green color
# upper_green = np.array([80, 255, 255])  # Upper bound of green color

# # Create a mask where green colors are white and other colors are black
# mask = cv2.inRange(hsv, lower_green, upper_green)

# # Isolate the green boxes from the image with boxes
# green_boxes = cv2.bitwise_or(image_with_boxes, image_with_boxes, mask = mask )
# plt.imshow(green_boxes)
# plt.show()
# # Add the green boxes to the original image
# result = cv2.addWeighted(original_image, 1, green_boxes[1], 1, 0)

# # Save or display the result
# cv2.imwrite('/content/original_with_green_boxes.png', result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# import cv2
# import numpy as np

# # Load the original image and the image with green boxes
# original_image = cv2.imread('/content/0-0_jpg.rf.91a1b5ecfa392c10b1986ad8070ffb2b11111.jpg')  # Path to the original image
# image_with_boxes = cv2.imread('/content/output_image.png')  # Path to the image with green boxes

# # Convert the image with green boxes to HSV (Hue, Saturation, Value) color space
# hsv = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2HSV)

# # Define the range for green color in HSV space
# lower_green = np.array([40, 40, 40])  # Lower bound of green color
# upper_green = np.array([80, 255, 255])  # Upper bound of green color

# # Create a mask where green colors are white and other colors are black
# mask = cv2.inRange(hsv, lower_green, upper_green)

# # Invert the mask to get the non-green areas (green areas become black, others become white)
# mask_inv = cv2.bitwise_not(mask)

# # Use the inverted mask to extract the non-green parts from the image with boxes
# non_green_parts = cv2.bitwise_and(image_with_boxes, image_with_boxes, mask=mask_inv)

# # Use the mask to extract the green parts from the original image
# green_parts = cv2.bitwise_and(original_image, original_image, mask=mask)

# # Combine the non-green parts of the image with boxes with the green parts from the original image
# result = cv2.add(non_green_parts, green_parts)

# # Save or display the result
# cv2.imwrite('/content/output_image.jpg', result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# cnts = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = cnts[0] if len(cnts) == 2 else cnts[1]
# cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])
# color = (0, 255, 0)  # Green color
# tw = 0
# start_h = 0
# start_x = start_y = 0
# for index,c in enumerate(cnts):
#   x,y,w,h = cv2.boundingRect(c)
#   if index != 0:
#     print("y:",y,"h:",h,"x:",x)
#     # if start_h >= h :
#     image[y:y+h, x:x+w+19] = color

# #         tw = tw + w
# #     else:
# #         cv2.rectangle(image, (start_x, y), (start_x + tw, y + h), color, 2)
# #         start_h = h
# #         start_x = x
# #     # print("y:",y,"prev_h:",prev_h,"start_h:",start_h,"start_x",start_x)
# #   #     if flag == 0 and prev_h < h :
# #   #       flag = 1
# #   #       break
# #   #     else:
# #   #       flag = 0

# #     # image[y:y+h+int(average_height), x:x+w+int(average_width)] = color
# # # cnts = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# # # cnts = cnts[0] if len(cnts) == 2 else cnts[1]
# # # cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

# # # color = (0, 255, 0)  # Green color
# # # start_y = 0
# # # prev_h = 0
# # # prev_y = 0
# # # for index, c in enumerate(cnts):
# # #       x, y, w, h = cv2.boundingRect(c)
# # #       print("y",y,"h",h,"ph:",prev_h,"sy",start_y,"py:",prev_y)
# # #       if index != 0:
# # #           if prev_h < h:
# # #             # if prev_h != 0:
# # #             cv2.rectangle(image, (x, start_y), (x + w, prev_y + prev_h), color, 2)
# # #             start_y = prev_y + prev_h
# # #             prev_h = h
# # #             prev_y = y
# # #           else:
# # #             if prev_h >= h:
# # #               prev_h = h
# # #               prev_y = y
# # #             # cv2.rectangle(image, (x, start_y), (x + w, prev_y + prev_h), (0, 0, 255), 2)
# # #           # prev_h = h
# # #           if index == 20:
# # #             # cv2.rectangle(image, (x, start_y), (x + w, prev_y + prev_h), color, 2)
# # #             break





# cv2.imwrite("/content/temp_rect1.png",image)

In [ ]:
# import cv2
# import numpy as np

# # Load the image
# # image = cv2.imread('/content/temp_rect1.png')
# image = cv2.imread('/content/temp_thres_white.png')
# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # # Threshold to detect green regions
# # lower_green = np.array([0, 150, 0])
# # upper_green = np.array([0, 255, 0])
# # mask_green = cv2.inRange(image, lower_green, upper_green)

# # # Find contours in the green regions
# # contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# contours = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# contours = contours[0] if len(contours) == 2 else contours[1]
# contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])

# # Separate contours into left and right columns
# left_contours = []
# right_contours = []

# image_center = image.shape[1] // 2

# for cnt in contours:
#     x, y, w, h = cv2.boundingRect(cnt)
#     if x < image_center:
#         left_contours.append((x, y, w, h))
#     else:
#         right_contours.append((x, y, w, h))

# # Draw right column contours first
# for (x, y, w, h) in right_contours:
#     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color

# # Draw left column contours and adjust them if they intersect with right column boxes
# for (lx, ly, lw, lh) in left_contours:
#     for (rx, ry, rw, rh) in right_contours:###########################
#         if lx + lw > rx and ly < ry + rh and ly + lh > ry:  # Check for intersection
#             lw = rx - lx  # Adjust left contour width to stop at right contour

#     cv2.rectangle(image, (lx, ly), (lx + lw, ly + lh), (0, 0, 255), 2)  # Red color

# # Display the image with modified rectangles
# cv2.imwrite("/content/temp_gridded_c1.png",image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# import cv2
# import numpy as np

# # Load the image
# # image = cv2.imread('/content/temp_rect1.png')
# image = cv2.imread('/content/temp_thres_black.png')
# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # # Threshold to detect green regions
# # lower_green = np.array([0, 150, 0])
# # upper_green = np.array([0, 255, 0])
# # mask_green = cv2.inRange(image, lower_green, upper_green)

# # # Find contours in the green regions
# # contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# contours = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# contours = contours[0] if len(contours) == 2 else contours[1]
# contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])

# # Separate contours into left and right columns
# left_contours = []
# right_contours = []

# image_center = image.shape[1] // 2

# for cnt in contours:
#     x, y, w, h = cv2.boundingRect(cnt)
#     if x < image_center:
#         left_contours.append((x, y, w, h))
#     else:
#         right_contours.append((x, y, w, h))

# # Draw right column contours first
# for (x, y, w, h) in right_contours:
#     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color

# # Draw left column contours and adjust them if they intersect with right column boxes
# for (lx, ly, lw, lh) in left_contours:
#     for (rx, ry, rw, rh) in right_contours:###########################
#         if lx + lw > rx and ly < ry + rh and ly + lh > ry:  # Check for intersection
#             lw = rx - lx  # Adjust left contour width to stop at right contour

#     cv2.rectangle(image, (lx, ly), (lx + lw, ly + lh), (0, 0, 255), 2)  # Red color

# # Display the image with modified rectangles
# cv2.imwrite("/content/temp_gridded_c2.png",image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# cnts = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = cnts[0] if len(cnts) == 2 else cnts[1]
# cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])
# color = (0, 255, 0)  # Green color

# for index,c in enumerate(cnts):
#   x,y,w,h = cv2.boundingRect(c)
#   if index != 0:

#     # image[y:y+h+int(h/average_height), x:x+w+int(w/average_width)] = color

#     image[y:y+h+10, x:x+w+20] = color
# cv2.imwrite("/content/temp_rect2.png",image)

In [ ]:
# import cv2
# import numpy as np

# # Load the image
# image = cv2.imread('/content/temp_rect2.png')

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Threshold to detect green regions
# lower_green = np.array([0, 150, 0])
# upper_green = np.array([0, 255, 0])
# mask_green = cv2.inRange(image, lower_green, upper_green)

# # Find contours in the green regions
# contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Separate contours into left and right columns
# left_contours = []
# right_contours = []

# image_center = image.shape[1] // 2

# for cnt in contours:
#     x, y, w, h = cv2.boundingRect(cnt)
#     if x < image_center:
#         left_contours.append((x, y, w, h))
#     else:
#         right_contours.append((x, y, w, h))

# # Draw right column contours first
# for (x, y, w, h) in right_contours:
#     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color

# # Draw left column contours and adjust them if they intersect with right column boxes
# for (lx, ly, lw, lh) in left_contours:
#     for (rx, ry, rw, rh) in right_contours:
#         if lx + lw > rx and ly < ry + rh and ly + lh > ry:  # Check for intersection
#             lw = rx - lx  # Adjust left contour width to stop at right contour

#     cv2.rectangle(image, (lx, ly), (lx + lw, ly + lh), (0, 0, 255), 2)  # Red color

# # Display the image with modified rectangles
# cv2.imwrite("/content/temp_gridded.png",image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# import cv2
# import numpy as np

# # Load the original image (before green overlay)
# original_image = cv2.imread("/content/0-0_jpg.rf.91a1b5ecfa392c10b1986ad8070ffb2b11111.jpg")

# # Load the modified image (with green overlay and red boxes)
# image_with_green = cv2.imread('/content/temp_gridded.png')

# # Convert the image to grayscale
# gray = cv2.cvtColor(image_with_green, cv2.COLOR_BGR2GRAY)

# # Threshold to detect green regions
# lower_green = np.array([0, 150, 0])
# upper_green = np.array([0, 255, 0])
# mask_green = cv2.inRange(image_with_green, lower_green, upper_green)

# # Find contours in the green regions
# contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Separate contours into left and right columns
# left_contours = []
# right_contours = []

# image_center = image_with_green.shape[1] // 2

# for cnt in contours:
#     x, y, w, h = cv2.boundingRect(cnt)
#     if x < image_center:
#         left_contours.append((x, y, w, h))
#     else:
#         right_contours.append((x, y, w, h))

# # Draw right column contours first and remove green color by restoring the original image
# for (x, y, w, h) in right_contours:
#     cv2.rectangle(image_with_green, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color
#     # Restore the original image in the green regions
#     image_with_green[y:y+h, x:x+w][mask_green[y:y+h, x:x+w] == 255] = original_image[y:y+h, x:x+w][mask_green[y:y+h, x:x+w] == 255]

# # Draw left column contours, adjust them, and remove green color by restoring the original image
# for (lx, ly, lw, lh) in left_contours:
#     for (rx, ry, rw, rh) in right_contours:
#         if lx + lw > rx and ly < ry + rh and ly + lh > ry:  # Check for intersection
#             lw = rx - lx  # Adjust left contour width to stop at right contour

#     cv2.rectangle(image_with_green, (lx, ly), (lx + lw, ly + lh), (0, 0, 255), 2)  # Red color
#     # Restore the original image in the green regions
#     image_with_green[ly:ly+lh, lx:lx+lw][mask_green[ly:ly+lh, lx:lx+lw] == 255] = original_image[ly:ly+lh, lx:lx+lw][mask_green[ly:ly+lh, lx:lx+lw] == 255]

# # Display the image with the green regions removed and text visible
# # cv2.imshow("Restored Image", image_with_green)
# cv2.imwrite("/content/temp_gridded.png",image_with_green)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# temp_gridded_c1 = cv2.imread("/content/temp_gridded_c1.png")
# temp_gridded_c2 = cv2.imread("/content/temp_gridded_c2.png")
# original_image = cv2.imread("/content/0-0_jpg.rf.91a1b5ecfa392c10b1986ad8070ffb2b11111.jpg")
# combined_gridded = cv2.bitwise_or(temp_gridded_c1, temp_gridded_c2)
# cv2.imwrite("/content/combined_gridded.png",combined_gridded)
# cv2.imwrite("/content/combined_gridded.png",image)

# **Text take out**